In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

### Convert to mg/L

$\frac{mg}{L}: y_{CO_2}\frac{𝑃·𝑀𝑟}{𝑅·𝑇}$

where:

$P: atm$

$Mr: g/mol$

$T: K$

$R: atm~m^3/mol/K$

In [ ]:
P = 1 # atm
Mr = 44.01 # g/mol for CO2
T = 298.15 # K
R = 8.205e-5 # atm.m3/mol/K

In [ ]:
# create a dataframe with experimental results - x: Molar ratio of co2 [dimensionless] and y: Adsorption capacity [mg/g] 
df = pd.DataFrame({'x': [0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1],
                 'y': [52.64, 72.59, 99.67, 141.79, 182.48, 203.68, 203.56, 204.33, 204.90]})


# define x and y column
xdata = df.loc[:, 'x']*P*Mr/R/T # mg/L
ydata = df.loc[:, 'y']

xtest = np.linspace(min(xdata), max(xdata), 50)

"""Since you have two (2 or 3) parameters in the model, we can pick two (2 or 3) data points from the data pool and plug these 
two (2 or 3) data points into the model and solve the values of these three parameters. 
The solution can be used as initial guess"""

**Chemical adsorption isotherm models - Langmuir**

In [ ]:
param_names = ['k_l in (L/mg)', 'q_max in (mg/g)']

langmuir = lambda x, *p: p[0]*p[1]*x/(1+p[0]*x)
     
langmuir_params, pcov = curve_fit(langmuir, 
                         xdata, 
                         ydata,
                         [1, 200],
                         bounds=(0, [500, 500]))

print('The values of', param_names, 'are', langmuir_params)

fig, ax = plt.subplots(ncols = 2, figsize = (10,4))
ax[0].plot(xdata, ydata, 'ko', mfc = 'none', label = 'Observed')
ax[0].plot(xtest, langmuir(xtest, *langmuir_params), 'k--', mfc = 'none', label = 'Predicted')
ax[0].set_xlabel("C$_e$ [mg/L]", fontsize = 10, fontweight = 'bold')
ax[0].set_ylabel("q$_e$ [mg/g]", fontsize = 10, fontweight = 'bold')
ax[0].legend()
ax[0].set_title('Langmuir fit')

ax[1].axhline(y = 0.0, color = 'k', linestyle = '--')
ax[1].plot(ydata, ydata - langmuir(xdata, *langmuir_params), 'ko', mfc = 'none')
ax[1].set_xlabel('y$_{predicted}$', fontsize = 10, fontweight = "bold")
ax[1].set_ylabel('residuals', fontsize = 10, fontweight = "bold")


**Adsorption empirical isotherm models - Freundlich, Temkin, Toth & Sips**

In [ ]:
param_names = ['k_f in (L^(1/n_f)·mg^(1-1/n_f)·g^-1)', 'n_f']

freundlich = lambda x, *p: p[0]*x**(1/p[1])

freundlich_params, pcov = curve_fit(freundlich, 
                         xdata, 
                         ydata,
                         p0 = [50, 2],
                         bounds=(0, [np.inf, np.inf]))

print('The values of', param_names, 'are', freundlich_params)

fig, ax = plt.subplots(ncols = 2, figsize = (10,4))
ax[0].plot(xdata, ydata, 'ko', mfc = 'none', label = 'Observed')
ax[0].plot(xtest, freundlich(xtest, *freundlich_params), 'k--', mfc = 'none', label = 'Predicted')
ax[0].set_xlabel("C$_e$ [mg/L]", fontsize = 10, fontweight = 'bold')
ax[0].set_ylabel("q$_e$ [mg/g]", fontsize = 10, fontweight = 'bold')
ax[0].legend()
ax[0].set_title('Freundlich fit')

ax[1].axhline(y = 0.0, color = 'k', linestyle = '--')
ax[1].plot(ydata, ydata - freundlich(xdata, *freundlich_params), 'ko', mfc = 'none')
ax[1].set_xlabel('y$_{predicted}$', fontsize = 10, fontweight = "bold")
ax[1].set_ylabel('residuals', fontsize = 10, fontweight = "bold")

In [ ]:
param_names = ['a_T in (L/mg)', 'b_T in (J/mol)']

temkin = lambda x, *p: (8.314*298.15/p[1])*(np.log(p[0]*x))
        
temkin_params, pcov = curve_fit(temkin,
                         xdata,
                         ydata,
                         p0 = [100, 100],
                         bounds=(0, [500, 500]))

print('The values of', param_names, 'are', temkin_params)

fig, ax = plt.subplots(ncols = 2, figsize = (10,4))
ax[0].plot(xdata, ydata, 'ko', mfc = 'none', label = 'Observed')
ax[0].plot(xtest, temkin(xtest, *temkin_params), 'k--', mfc = 'none', label = 'Predicted')
ax[0].set_xlabel("C$_e$ [mg/L]", fontsize = 10, fontweight = 'bold')
ax[0].set_ylabel("q$_e$ [mg/g]", fontsize = 10, fontweight = 'bold')
ax[0].legend()
ax[0].set_title('Temkin fit')

ax[1].axhline(y = 0.0, color = 'k', linestyle = '--')
ax[1].plot(ydata, ydata - temkin(xdata, *temkin_params), 'ko', mfc = 'none')
ax[1].set_xlabel('y$_{predicted}$', fontsize = 10, fontweight = "bold")
ax[1].set_ylabel('residuals', fontsize = 10, fontweight = "bold")

In [ ]:
param_names = ['k_toth in (mg/g)', 'a_toth in (mg^z·L^-z)', 'z (dimensionless)']

toth = lambda x, *p: p[0]*x/(p[1] + x**p[2])**(1/p[2])

sol_root = optimize.root(lambda p: toth(xdata[:3], *p), [10, 1e-1, 1])

print('An estimation of initial values is', sol_root.x)
        
toth_params, pcov = curve_fit(toth,
                         xdata,
                         ydata,
                         p0 = sol_root.x,
                         bounds=(0, [1e5, 1e5, 2]))

print('The values of', param_names, 'are', toth_params)

fig, ax = plt.subplots(ncols = 2, figsize = (8,3), constrained_layout = True)
ax[0].plot(xdata, ydata, 'ko', mfc = 'none', label = 'Observed')
ax[0].plot(xtest, toth(xtest, *toth_params), 'k--', mfc = 'none', label = 'Predicted')
ax[0].set_xlabel("C$_e$ [mg/L]", fontsize=10, fontweight='bold')
ax[0].set_ylabel("q$_e$ [mg/g]", fontsize=10, fontweight='bold')
ax[0].legend()
ax[0].set_title('Toth fit')

ax[1].axhline(y = 0.0, color = 'k', linestyle = '--')
ax[1].plot(toth(xdata, *toth_params), ydata - toth(xdata, *toth_params), 'ko', mfc = 'none')
ax[1].set_xlabel('y$_{predicted}$', fontsize = 10, fontweight = "bold")
ax[1].set_ylabel('residuals', fontsize = 10, fontweight = "bold")

In [ ]:
param_names = ['q_ms in (mg/g)', 'K_s in (L^ns·mg^-ns)', 'ns (dimensionless)']

sips = lambda x, *p: p[0]*p[1]*x**p[2]/(1+p[1]*x**p[2])
  
sips_params, pcov = curve_fit(sips, 
                         xdata, 
                         ydata,
                         p0 = [2, 0.01, 0.01],
                         bounds=(0, [np.inf, np.inf, np.inf]))

print('The values of', param_names, 'are', sips_params)

fig, ax = plt.subplots(ncols = 2, figsize = (8,3), constrained_layout = True)
ax[0].plot(xdata, ydata, 'ko', mfc = 'none', label = 'Observed')
ax[0].plot(xtest, sips(xtest, *sips_params), 'k--', mfc = 'none', label = 'Predicted')
ax[0].set_xlabel("C$_e$ [mg/L]", fontsize=10, fontweight='bold')
ax[0].set_ylabel("q$_e$ [mg/g]", fontsize=10, fontweight='bold')
ax[0].legend()
ax[0].set_title('Sips fit')

ax[1].axhline(y = 0.0, color = 'k', linestyle = '--')
ax[1].plot(sips(xdata, *sips_params), ydata - sips(xdata, *sips_params), 'ko', mfc = 'none')
ax[1].set_xlabel('y$_{predicted}$', fontsize = 10, fontweight = "bold")
ax[1].set_ylabel('residuals', fontsize = 10, fontweight = "bold")

**Adsorption models based on the Polanyi’s potential theory - DR**

In [ ]:
param_names = ['k_dr in (mol^2·J^–2)', 'qmax in (mg/g)']

dr = lambda x, *p: p[1]*np.exp(-p[0]*(8.314*298.15)**2*np.log(1+1/x)**2)
        
dr_params, pcov = curve_fit(dr, 
                         xdata, 
                         ydata,
                         p0 = [1e-5, 100],
                         bounds=(0, [np.inf, np.inf]))

print('The values of', param_names, 'are', dr_params)

fig, ax = plt.subplots(ncols = 2, figsize = (10,4))
ax[0].plot(xdata, ydata, 'ko', mfc = 'none', label = 'Observed')
ax[0].plot(xtest, dr(xtest, *dr_params), 'k--', mfc = 'none', label = 'Predicted')
ax[0].set_xlabel("C$_e$ [mg/L]", fontsize = 10, fontweight='bold')
ax[0].set_ylabel("q$_e$ [mg/g]", fontsize = 10, fontweight='bold')
ax[0].legend()
ax[0].set_title('DR fit')

ax[1].axhline(y = 0.0, color = 'k', linestyle = '--')
ax[1].plot(ydata, ydata - dr(xdata, *dr_params), 'ko', mfc = 'none')
ax[1].set_xlabel('y$_{predicted}$', fontsize = 10, fontweight = "bold")
ax[1].set_ylabel('residuals', fontsize = 10, fontweight = "bold")

In [ ]:
fig, ax = plt.subplots(ncols = 2, figsize = (10,4), constrained_layout = True)
ax[0].plot(xdata, ydata, 'ko', mfc = 'none', label = 'Observed')
ax[0].plot(xtest, langmuir(xtest, *langmuir_params), 'r--', mfc = 'none', label = 'Langmuir')
ax[0].plot(xtest, freundlich(xtest, *freundlich_params), 'g--', mfc = 'none', label = 'Freundlich')
ax[0].plot(xtest, temkin(xtest, *temkin_params), 'b--', mfc = 'none', label = 'Temkin')
ax[0].plot(xtest, toth(xtest, *toth_params), 'm--', mfc = 'none', label = 'Toth')
ax[0].plot(xtest, sips(xtest, *sips_params), 'c--', mfc = 'none', label = 'Sips')
ax[0].plot(xtest, dr(xtest, *dr_params), 'y--', mfc = 'none', label = 'DR')

ax[0].set_xlabel("C$_e$ [mg/L]", fontsize=10, fontweight='bold')
ax[0].set_ylabel("q$_e$ [mg/g]", fontsize=10, fontweight='bold')
ax[0].legend()
ax[0].set_title('All together fit')
ax[0].grid(linestyle = ':')

ax[1].axhline(y = 0.0, color = 'k', linestyle = '--')
ax[1].plot(langmuir(xdata, *langmuir_params), ydata - langmuir(xdata, *langmuir_params), 'ro', mfc = 'none')
ax[1].plot(freundlich(xdata, *freundlich_params), ydata - freundlich(xdata, *freundlich_params), 'go', mfc = 'none')
ax[1].plot(temkin(xdata, *temkin_params), ydata - temkin(xdata, *temkin_params), 'bo', mfc = 'none')
ax[1].plot(toth(xdata, *toth_params), ydata - toth(xdata, *toth_params), 'mo', mfc = 'none')
ax[1].plot(sips(xdata, *sips_params), ydata - sips(xdata, *sips_params), 'co', mfc = 'none')
ax[1].plot(dr(xdata, *dr_params), ydata - dr(xdata, *dr_params), 'yo', mfc = 'none')

ax[1].set_xlabel('y$_{predicted}$', fontsize = 10, fontweight = "bold")
ax[1].set_ylabel('residuals', fontsize = 10, fontweight = "bold")
ax[1].grid(linestyle = ':')
#ax[1].set_xticks([])